### This script converts position data from the DVL to lines of XML keypoints defining the trajectory

In [1]:
import pandas as pd
df = pd.read_csv("dvl_position.csv")
df.head()

,t,__msgtype__,format,header.__msgtype__,header.frame_id,header.stamp.__msgtype__,header.stamp.nanosec,header.stamp.sec,pitch,roll,status,std,t_last_reset,time,x,y,yaw,z
0,1.724164e+09,sensors/msg/DVLPosition,json_v3.1,std_msgs/msg/Header,NaN,builtin_interfaces/msg/Time,801519155,1724164479,1.590327,-2.002819,0,0.0,1.724164e+09,1.687100e+09,0.051770,-0.014073,-0.632555,0.010189
1,1.724164e+09,sensors/msg/DVLPosition,json_v3.1,std_msgs/msg/Header,NaN,builtin_interfaces/msg/Time,6940126,1724164480,1.843149,-1.583715,0,0.0,1.724164e+09,1.687100e+09,0.064422,-0.009308,-0.610183,0.025066
2,1.724164e+09,sensors/msg/DVLPosition,json_v3.1,std_msgs/msg/Header,NaN,builtin_interfaces/msg/Time,221364498,1724164480,2.150046,-1.233694,0,0.0,1.724164e+09,1.687100e+09,0.073586,0.002362,-0.764366,0.047654
3,1.724164e+09,sensors/msg/DVLPosition,json_v3.1,std_msgs/msg/Header,NaN,builtin_interfaces/msg/Time,452746391,1724164480,2.295287,-0.836874,0,0.0,1.724164e+09,1.687100e+09,0.085261,0.014252,-1.169754,0.079026
4,1.724164e+09,sensors/msg/DVLPosition,json_v3.1,std_msgs/msg/Header,NaN,builtin_interfaces/msg/Time,660883903,1724164480,2.349584,-0.550021,0,0.0,1.724164e+09,1.687100e+09,0.100592,0.023341,-1.343815,0.113974


In [2]:
import numpy as np

# Parameters
input_file = "dvl_position.csv"
output_file = "keypoints.txt"
average_every = 10
xyz_shift = (0.0, 0.0, 0.0)
rpy_shift = (0.0, 0.0, 0.0)
convert_to_radians = True
time_start_at_zero = True

# Read the data
df = pd.read_csv(input_file)
usable_len = (len(df) // average_every) * average_every
df = df.iloc[:usable_len].copy()

# Group and average
groups = df.groupby(df.index // average_every, sort=True).mean(numeric_only=True)

if time_start_at_zero:
    groups["t"] -= groups["t"].iloc[0]

if convert_to_radians:
    groups["roll"] = np.deg2rad(groups["roll"])
    groups["pitch"] = np.deg2rad(groups["pitch"])
    groups["yaw"] = np.deg2rad(groups["yaw"])

# add shift
groups["x"] += xyz_shift[0]
groups["y"] += xyz_shift[1]
groups["z"] += xyz_shift[2]
groups["roll"] += rpy_shift[0]
groups["pitch"] += rpy_shift[1]
groups["yaw"] += rpy_shift[2]

# make keypoints of precition 4
fmt = f"{{:.{4}f}}" # num of decimals
lines = []
for _, row in groups.iterrows():
    t = fmt.format(row["t"])
    # Join with a space
    xyz = " ".join(fmt.format(v) for v in (row["x"], row["y"], row["z"]))
    rpy = " ".join(fmt.format(v) for v in (row["roll"], row["pitch"], row["yaw"]))
    lines.append(f'<keypoint time="{t}" xyz="{xyz}" rpy="{rpy}"/>')

out = "\n".join(lines)

# To file
with open(output_file, "w") as f:
    f.write(out)